In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import os

import flax
from flax import jax_utils
import jax
import jax.numpy as jnp
import numpy as np
import optax
import torch
import tensorflow as tf
import tqdm

import init2winit
import fastmri

import i2w

from fastmri.models import unet as t_unet
from fastmri.pl_modules import data_module
from fastmri.pl_modules import unet_module
from fastmri.data.transforms import UnetDataTransform
from fastmri.data.subsample import create_mask_for_mask_type

from init2winit.model_lib import unet as f_unet
from init2winit.dataset_lib import fastmri_dataset
from init2winit.dataset_lib import data_utils
from init2winit.optimizer_lib import optimizers

jax.devices()

[GpuDevice(id=0, process_index=0),
 GpuDevice(id=1, process_index=0),
 GpuDevice(id=2, process_index=0),
 GpuDevice(id=3, process_index=0),
 GpuDevice(id=4, process_index=0),
 GpuDevice(id=5, process_index=0),
 GpuDevice(id=6, process_index=0),
 GpuDevice(id=7, process_index=0)]

# U-Net parameters

## `fastmri`

In [2]:
t_model = t_unet.Unet(in_chans=1, out_chans=1, chans=32, num_pool_layers=4, drop_prob=0.0)

## `init2winit`

In [3]:
f_model = f_unet.UNet(out_chans=1, chans=32, num_pool_layers=4, drop_prob=0.0)
f_params = i2w.convert_params(t_model, f_model, batch_size=1)
f_params8 = i2w.convert_params(t_model, f_model, batch_size=8)

Abs sum diff: 0.08570768
Abs sum diff: 2233292.0


## Compared

In [4]:
jax.tree_map(jnp.allclose, f_params8, f_params)

{'params': {'ConvBlock_0': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel': DeviceArray(True, dtype=bool)}},
  'ConvBlock_1': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel': DeviceArray(True, dtype=bool)}},
  'ConvBlock_2': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel': DeviceArray(True, dtype=bool)}},
  'ConvBlock_3': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel': DeviceArray(True, dtype=bool)}},
  'ConvBlock_4': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel': DeviceArray(True, dtype=bool)}},
  'ConvBlock_5': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel': DeviceArray(True, dtype=bool)}},
  'ConvBlock_6': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel': DeviceArray(True, dtype=bool)}},
  'ConvBlock_7': {'Conv_0': {'kernel': DeviceArray(True, dtype=bool)},
   'Conv_1': {'kernel':